# 🏋️ Azure AI Search + Semantic Kernel + AI Agents: Fitness-Fun Workshop 🤸

Welcome to this self-guided workshop where you'll:

1. **Create** an Azure AI Search index containing some sample fitness equipment data
2. **Upload** and verify your documents
3. **Create** a Semantic Kernel Agent (powered by Azure AI Agent Service) that uses the Azure AI Search tool
4. **Run** an asynchronous conversation to query your index (with a fun fitness twist)

> **Note:** This demo uses Semantic Kernel’s abstractions over Azure AI Agents. Make sure you run:
>
> ```bash
> pip install semantic-kernel[azure]
> ```

Also ensure you’ve set your environment variables for:

- `PROJECT_CONNECTION_STRING`
- `MODEL_DEPLOYMENT_NAME`

Let’s get started!

## Prerequisites

Before running the cells below, please verify:

1. You have installed the required dependency:
   ```bash
   pip install semantic-kernel[azure]
   ```
2. Your environment is configured with the necessary variables (`PROJECT_CONNECTION_STRING` and `MODEL_DEPLOYMENT_NAME`).

In [7]:
# Uncomment and run the cell below if you have not installed the dependency yet
# !pip install semantic-kernel[azure]

## 1. Create & Populate Azure AI Search Index

In this section we will:

1. **Create** an Azure AI Search index called `myfitnessindex` with a schema suited for fitness items
2. **Upload** sample documents containing fitness equipment data
3. **Verify** that the documents are searchable

Make sure your environment has the appropriate search credentials (typically obtained via your AI Foundry project).

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField
from azure.search.documents import SearchClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType

# Load environment variables
notebook_path = Path().absolute()
env_path = notebook_path.parent.parent / '.env'  # Adjust path as needed
load_dotenv(env_path)

connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
if not connection_string:
    raise ValueError("🚨 PROJECT_CONNECTION_STRING not set in .env.")

# Initialize the AI Project client to access project resources
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string
    )
    print("✅ Initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing AIProjectClient: {e}")

# Get the Azure AI Search connection details from the project (including endpoint and API key)
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH,
    include_credentials=True
)

if not search_conn:
    raise RuntimeError("❌ No default Azure AI Search connection found in your project.")

# Define the index name for our fitness data
index_name = "myfitnessindex"

try:
    credential = AzureKeyCredential(search_conn.key)
    index_client = SearchIndexClient(endpoint=search_conn.endpoint_url, credential=credential)
    print("✅ Created SearchIndexClient")
    
    search_client = SearchClient(
        endpoint=search_conn.endpoint_url,
        index_name=index_name,
        credential=credential
    )
    print("✅ Created SearchClient for document operations")
except Exception as e:
    print(f"❌ Error creating search clients: {e}")

### Define the Index Schema

We will create an index with the following fields:

- `FitnessItemID`: Unique key
- `Name`: Searchable text field (also filterable)
- `Category`: Searchable, filterable, and facetable (e.g. Strength, Cardio, Flexibility)
- `Price`: Numeric field (filterable, sortable, and facetable)
- `Description`: Full-text searchable field

In [ ]:
def create_fitness_index():
    fields = [
        SimpleField(name="FitnessItemID", type=SearchFieldDataType.String, key=True),
        SearchableField(name="Name", type=SearchFieldDataType.String, filterable=True),
        SearchableField(name="Category", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SimpleField(name="Price", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String)
    ]

    index = SearchIndex(name=index_name, fields=fields)

    # Delete the index if it already exists (for a fresh start)
    if index_name in [x.name for x in index_client.list_indexes()]:
        index_client.delete_index(index_name)
        print(f"🗑️ Deleted existing index: {index_name}")

    created = index_client.create_index(index)
    print(f"🎉 Created index: {created.name}")

# Create the index
create_fitness_index()

### Upload Sample Documents

Now we’ll add some sample fitness items to `myfitnessindex`.

In [ ]:
def upload_fitness_docs():
    search_client = SearchClient(
        endpoint=search_conn.endpoint_url,
        index_name=index_name,
        credential=AzureKeyCredential(search_conn.key)
    )

    sample_docs = [
        {
            "FitnessItemID": "1",
            "Name": "Adjustable Dumbbell",
            "Category": "Strength",
            "Price": 59.99,
            "Description": "A compact, adjustable weight for targeted muscle workouts."
        },
        {
            "FitnessItemID": "2",
            "Name": "Yoga Mat",
            "Category": "Flexibility",
            "Price": 25.0,
            "Description": "Non-slip mat designed for yoga, Pilates, and other exercises."
        },
        {
            "FitnessItemID": "3",
            "Name": "Treadmill",
            "Category": "Cardio",
            "Price": 499.0,
            "Description": "A sturdy treadmill with adjustable speed and incline settings."
        },
        {
            "FitnessItemID": "4",
            "Name": "Resistance Bands",
            "Category": "Strength",
            "Price": 15.0,
            "Description": "Set of colorful bands for light to moderate resistance workouts."
        }
    ]

    result = search_client.upload_documents(documents=sample_docs)
    print(f"🚀 Upload result: {result}")

upload_fitness_docs()
print("✅ Documents uploaded to search index")

### Verify the Documents

Let’s perform a basic search query (e.g. for items in the **Strength** category) to ensure everything is working.

In [ ]:
results = search_client.search(search_text="Strength", filter=None, top=10)

print("🔍 Search results for 'Strength':")
print("-" * 50)
found_items = False
for doc in results:
    found_items = True
    print(f"Name: {doc['Name']}")
    print(f"Category: {doc['Category']}")
    print(f"Price: ${doc['Price']:.2f}")
    print(f"Description: {doc['Description']}")
    print("-" * 50)

if not found_items:
    print("No matching items found.")

## 2. Create Semantic Kernel Agent with Azure AI Search Tool

In this section we’ll use Semantic Kernel’s Azure AI Agent abstractions to build a fitness shopping assistant. This agent will:

- Use your Azure OpenAI model (specified by `MODEL_DEPLOYMENT_NAME`)
- Attach an **Azure AI Search tool** (pointing to `myfitnessindex`)
- Engage in an asynchronous conversation that queries the index based on user input

The code below uses asynchronous Python (with `asyncio`) and Semantic Kernel classes from the `semantic_kernel` package.

In [ ]:
import asyncio
import logging

from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType
from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

logging.basicConfig(level=logging.WARNING)

# For this demo, we will use the same index name as before
AZURE_AI_SEARCH_INDEX_NAME = "myfitnessindex"

# Get required environment variables
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
project_connection_string = os.environ.get("PROJECT_CONNECTION_STRING")

if not model_deployment_name:
    raise ValueError("🚨 MODEL_DEPLOYMENT_NAME not set in .env")
if not project_connection_string:
    raise ValueError("🚨 PROJECT_CONNECTION_STRING not set in .env")

# Create agent settings with required parameters
ai_agent_settings = AzureAIAgentSettings.create(
    model_deployment_name=model_deployment_name,
    project_connection_string=project_connection_string
)

async with (
    DefaultAzureCredential() as creds,
    AIProjectClient.from_connection_string(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value()
    ) as client,
):
    # List available connections and find one of type Azure AI Search
    conn_list = await client.connections.list()
    ai_search_conn_id = ""
    for conn in conn_list:
        if conn.connection_type == ConnectionType.AZURE_AI_SEARCH:
            ai_search_conn_id = conn.id
            break

    if not ai_search_conn_id:
        print("❌ No Azure AI Search connection found.")
        raise ValueError("❌ No Azure AI Search connection found.")

    # Create the Azure AI Search tool pointing to our fitness index
    ai_search_tool = AzureAISearchTool(
        index_connection_id=ai_search_conn_id, 
        index_name=AZURE_AI_SEARCH_INDEX_NAME
    )

    # Create the agent definition with instructions for a fitness shopping assistant
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        instructions="""
            You are a Fitness Shopping Assistant. You help users find fitness equipment based on their queries.
            Always include a disclaimer that you are not providing medical advice.
        """,
        tools=ai_search_tool.definitions,
        tool_resources=ai_search_tool.resources,
        headers={"x-ms-enable-preview": "true"},
    )

    # Create the Semantic Kernel Azure AI Agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )

    # Create a new conversation thread
    thread = await client.agents.create_thread()
    print(f"📝 Created thread with ID: {thread.id}")

    # Define some example fitness queries
    user_queries = [
        "Which items are best for strength training?",
        "I need something for cardio under $300. Any suggestions?"
    ]

    try:
        for query in user_queries:
            # Add the user message
            await agent.add_chat_message(
                thread_id=thread.id,
                message=ChatMessageContent(role=AuthorRole.USER, content=query),
            )
            print(f"\n# User: {query}\n")

            # Invoke the agent and stream its response
            async for content in agent.invoke(thread_id=thread.id):
                if content.role != AuthorRole.TOOL:
                    print(f"# Agent: {content.content}\n")
    finally:
        # Clean up the conversation thread and agent
        await client.agents.delete_thread(thread.id)
        await client.agents.delete_agent(agent.id)
        print("🗑️ Cleaned up agent and thread")

## 3. Cleanup

For this demo we already clean up the agent and thread inside the async function. In case you want to remove the search index as well (for a fresh start), run the cell below.

In [ ]:
try:
    index_client.delete_index(index_name)
    print(f"🗑️ Deleted index {index_name}")
except Exception as e:
    print(f"Error deleting index: {e}")

# 🎉 Congrats!

You've successfully:

1. Created an Azure AI Search index and populated it with fitness data
2. Verified the data via a basic search query
3. Built and run a Semantic Kernel Agent that leverages Azure AI Search to answer natural language queries

Feel free to explore further enhancements (e.g. integrating more tools or advanced evaluation) and enjoy your journey with Azure AI Foundry and Semantic Kernel!